## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Victoria,HK,22.2855,114.1577,75.33,80,78,5.01,broken clouds
1,1,Busselton,AU,-33.6500,115.3333,49.80,90,35,6.87,scattered clouds
2,2,Atikokan,CA,48.7555,-91.6216,49.62,61,100,9.08,overcast clouds
3,3,Cidreira,BR,-30.1811,-50.2056,71.15,78,72,22.97,broken clouds
4,4,Albany,US,42.6001,-73.9662,57.94,93,99,7.99,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your vacation?'))
max_temp = float(input('What is the maximum temperature you would like for your vacation?'))

What is the minimum temperature you would like for your vacation?70
What is the maximum temperature you would like for your vacation?80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) &
                                       (city_data_df['Max Temp'] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Victoria,HK,22.2855,114.1577,75.33,80,78,5.01,broken clouds
3,3,Cidreira,BR,-30.1811,-50.2056,71.15,78,72,22.97,broken clouds
5,5,Puerto Ayora,EC,-0.7393,-90.3518,72.79,79,71,8.86,broken clouds
18,18,Rikitea,PF,-23.1203,-134.9692,74.77,85,86,21.70,light rain
22,22,Riyadh,SA,24.6877,46.7219,78.94,34,89,5.19,overcast clouds
23,23,San Cristobal,VE,7.7669,-72.2250,78.98,87,98,3.58,overcast clouds
31,31,Neiafu,TO,-18.6500,-173.9833,75.88,73,75,8.05,broken clouds
32,32,Treinta Y Tres,UY,-33.2333,-54.3833,74.84,75,99,9.17,overcast clouds
33,33,Darnah,LY,32.7670,22.6367,70.45,69,20,8.50,few clouds
35,35,Kapaa,US,22.0752,-159.3190,75.18,95,27,2.55,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                164
City                   164
Country                164
Lat                    164
Lng                    164
Max Temp               164
Humidity               164
Cloudiness             164
Wind Speed             164
Current Description    164
dtype: int64

In [34]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(inplace = True)

<ipython-input-34-faf8934ce0c9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df = preferred_cities_df.dropna(inplace = True)


In [10]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Victoria,HK,75.33,broken clouds,22.2855,114.1577,
3,Cidreira,BR,71.15,broken clouds,-30.1811,-50.2056,
5,Puerto Ayora,EC,72.79,broken clouds,-0.7393,-90.3518,
18,Rikitea,PF,74.77,light rain,-23.1203,-134.9692,
22,Riyadh,SA,78.94,overcast clouds,24.6877,46.7219,
23,San Cristobal,VE,78.98,overcast clouds,7.7669,-72.2250,
31,Neiafu,TO,75.88,broken clouds,-18.6500,-173.9833,
32,Treinta Y Tres,UY,74.84,overcast clouds,-33.2333,-54.3833,
33,Darnah,LY,70.45,few clouds,32.7670,22.6367,
35,Kapaa,US,75.18,scattered clouds,22.0752,-159.3190,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params = params).json() 
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [25]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Victoria,HK,75.33,broken clouds,22.2855,114.1577,Mini Hotel Central
3,Cidreira,BR,71.15,broken clouds,-30.1811,-50.2056,Hotel Castelo
5,Puerto Ayora,EC,72.79,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
18,Rikitea,PF,74.77,light rain,-23.1203,-134.9692,People ThankYou
22,Riyadh,SA,78.94,overcast clouds,24.6877,46.7219,"InterContinental Riyadh, an IHG Hotel"


In [27]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df['Hotel Name'].replace("",np.nan, inplace = True)
clean_hotel_df = hotel_df.dropna()

In [28]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Victoria,HK,75.33,broken clouds,22.2855,114.1577,Mini Hotel Central
3,Cidreira,BR,71.15,broken clouds,-30.1811,-50.2056,Hotel Castelo
5,Puerto Ayora,EC,72.79,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
18,Rikitea,PF,74.77,light rain,-23.1203,-134.9692,People ThankYou
22,Riyadh,SA,78.94,overcast clouds,24.6877,46.7219,"InterContinental Riyadh, an IHG Hotel"
23,San Cristobal,VE,78.98,overcast clouds,7.7669,-72.2250,Posada Villaven C.A.
31,Neiafu,TO,75.88,broken clouds,-18.6500,-173.9833,Mystic Sands Beachfront Accommodation
32,Treinta Y Tres,UY,74.84,overcast clouds,-33.2333,-54.3833,Jardines Apart-Hotel
33,Darnah,LY,70.45,few clouds,32.7670,22.6367,فندق الفردوس
35,Kapaa,US,75.18,scattered clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


In [30]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))